## File names here:

In [1]:
tissues_with_atac = ['CHO','Brain','Liver','Lung','Kidney','Spleen','BMDMwt','BMDM1hKLA']

## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']
tss_annotation = doc['tss_annotation']

In [2]:
from create_output import *

txn_f = "Results/tss_annotation/txn_df_02_tissues.p"
#expr_f = "Results/merged/samples.merge.peaksexpression.log10"
expr_f = "Results/merged/tissues.merge.peaksexpression.log10"
anno_f = "Results/tss_annotation_peaks/all_peaks_txn_df_02_maxinfo.tsv"
rank_func = rank_median_samples
atac_f = "../10_15_Results/Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort"
tss_f = "/data/isshamie/genome/picr_final/mRNA_final.peak"#tss_annotation
all_atac = "../10_15_Results/Results/ATAC_results/ATAC_merge.bed"



In [3]:

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 15)


In [ ]:
if not os.path.exists("Results/output"):
    os.mkdir("Results/output")

In [ ]:
%%time
# bed_df, meta_df = construct_peakID(txn_anno_f, expr_f, anno_f, rank_func,
#                      atac_f=None, col_name=None)

bed_df, meta_df = construct_all_peaks(txn_f, expr_f, anno_f, rank_func, tss_f, atac_f,
                        all_atac=all_atac, out_f="Results/output/TSS1")

27075it [14:54, 30.25it/s]


         Chr     Start       End Strand        Stat  \
ID                                                    
3    picr_54  10883074  10883337      +    3.991000   
4     picr_2  37427816  37427999      +  129.025650   
5   picr_136   2231719   2231926      -   16.201666   
6    picr_38   6314085   6314339      +   11.446400   
8    picr_95    461540    462075      -    6.742963   

                                         Parent files  Total subpeaks  \
ID                                                                      
3   f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...              20   
4   f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...              23   
5   f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...              24   
6   f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...              25   
8   f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...              27   

   f04_peaks/MiscOrgans_START1_and_MiscOrgans_startinput1  \
ID                                       

63521it [15:09, 69.83it/s]
60116it [40:06, 22.53it/s]

In [8]:
bed_df.head()

,Chr,Start,End,ID,Stat,Strand
p0@SERHL_gene_1_2_asmbl_2,picr_0,175,177,p0@SERHL_gene_1_2_asmbl_2,0.00000,+
p0@SERHL_gene_1_4_asmbl_11,picr_0,18148,18150,p0@SERHL_gene_1_4_asmbl_11,0.00000,+
p0@SERHL_gene_1_4_asmbl_12,picr_0,20786,20788,p0@SERHL_gene_1_4_asmbl_12,0.00000,+
p1@SERHL_gene_1_4_asmbl_14,picr_0,20916,21066,p1@SERHL_gene_1_4_asmbl_14,2.05576,+
p0@SERHL_gene_1_4_asmbl_15,picr_0,22503,22505,p0@SERHL_gene_1_4_asmbl_15,0.00000,+


In [9]:
meta_df.head()

,Tissues,cs,CHO ATAC Region,ID,Gene,Gene ID,Transcript,Is Experimental,has ATAC
p0@SERHL_gene_1_2_asmbl_2,,,,,SERHL,gene_1_2,asmbl_2,0,0
p0@SERHL_gene_1_4_asmbl_11,,,,,SERHL,gene_1_4,asmbl_11,0,0
p0@SERHL_gene_1_4_asmbl_12,,,,,SERHL,gene_1_4,asmbl_12,0,0
p1@SERHL_gene_1_4_asmbl_14,"Heart,Spleen,MiscOrgans,FemaleReproductive,CHO...",0b11,,160866,SERHL,gene_1_4,asmbl_14,1,0
p0@SERHL_gene_1_4_asmbl_15,,,,,SERHL,gene_1_4,asmbl_15,0,0


-----------------------------